# Мастерская. KPMI

### Описание

KPMI (Keys to Personal Mastery Inventory) — Ключи персонального мастерства.

Тест «Ключи персонального мастерства» предназначен для определения индивидуального поведенческого стиля личности. Является оригинальной отечественной разработкой на базе широко известного типологического опросника Майер-Бриггс.

Помимо традиционного описания психологических особенностей личности тест позволяет прогнозировать, в какой сфере деятельности личность с наибольшей вероятностью сможет реализоваться и работать с удовольствием.

### Цель проекта

С помощью моделей классического машинного обучения попробовать улучшить качество предсказания сферы деятельности в которой человек сможет максимально самореализоваться.

Из результатов ответа на вопрос о профессии выделить собственно профессии, на основе которых построить каталог сфер деятельности;

Для каждой сферы деятельности построить классификатор(ы?), который по результатам заполнения анкеты дает прогноз о том, в
каких сферах деятельности респондент сможет максимально самореализоваться.

### Данные

- ответы на 60 вопросов анкеты;
- ответ на вопрос "кем ты работаешь?";
- ответ на разметочный вопрос "моя работа позволяет мне самореализовываться";
- время ответа в секундах на каждый вопрос;
- расчитанный психотип;
- 4 дихотимии психотипа.

## Подготовка, загрузка, осмотр данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
import pymorphy2
from statsmodels.stats.outliers_influence import variance_inflation_factor

import re
from nltk.corpus import stopwords
import nltk

from collections import Counter

import pkg_resources
from symspellpy import SymSpell, Verbosity

Задаём глобальные переменные.

In [2]:
SEED = 314159 # случайное зерно

Изменим настройки отображения, сняв ограничение на количество строк и колонок.

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Прочитаем данные в файл.

In [4]:
df = pd.read_csv('kpmi.ru_190k.csv', low_memory=False)

Сделаем осмотр данных.

In [5]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194568 entries, 0 to 194567
Data columns (total 133 columns):
 #    Column      Non-Null Count   Dtype  
---   ------      --------------   -----  
 0    q1          194568 non-null  int64  
 1    t1          194568 non-null  float64
 2    q2          194567 non-null  float64
 3    t2          194567 non-null  float64
 4    q3          194567 non-null  float64
 5    t3          194567 non-null  float64
 6    q4          194567 non-null  float64
 7    t4          194567 non-null  float64
 8    q5          194567 non-null  float64
 9    t5          194566 non-null  float64
 10   q6          194567 non-null  float64
 11   t6          194567 non-null  float64
 12   q7          194567 non-null  float64
 13   t7          194567 non-null  float64
 14   q8          194567 non-null  float64
 15   t8          194567 non-null  float64
 16   q9          194567 non-null  float64
 17   t9          194567 non-null  float64
 18   q10         194567 non

In [6]:
df.describe()

,q1,t1,q2,t2,q3,t3,q4,t4,q5,t5,q6,t6,q7,t7,q8,t8,q9,t9,q10,t10,q11,t11,q12,t12,q13,t13,q14,t14,q15,t15,q16,t16,q17,t17,q18,t18,q19,t19,q20,t20,q21,t21,q22,t22,q23,t23,q24,t24,q25,t25,q26,t26,q27,t27,q28,t28,q29,t29,q30,t30,q31,t31,q32,t32,q33,t33,q34,t34,q35,t35,q36,t36,q37,t37,q38,t38,q39,t39,q40,t40,q41,t41,q42,t42,q43,t43,q44,t44,q45,t45,q46,t46,q47,t47,q48,t48,q49,t49,q50,t50,q51,t51,q52,t52,q53,t53,q54,t54,q55,t55,q56,t56,q57,t57,q58,t58,q59,t59,q60,t60,t61,q62,t62,scale_e,scale_i,scale_f,scale_t,scale_s,scale_n,scale_j,scale_p
count,194568.000000,1.945680e+05,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,194566.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194566.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.00000,194566.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,1.945670e+05,194567.000000,194567.000000,194567.000000,194567.000000,194567.000000,1.945660e+05,194567.000000,194566.000000,194567.000000,194567.000000,194567.000000,194566.000000,194567.000000,194567.000000,188316.000000,185189.000000,185189.000000,172164.000000,172164.000000,172164.000000,172164.000000,172164.000000,172164.000000,172164.000000,172164.000000
mean,1.549391,5.607079e+01,1.552745,21.275059,1.348610,3.076964e+01,1.382819,14.441525,1.471755,14.014786,1.672015,3.510847e+01,1.373013,18.764205,1.575971,4.177171e+01,1.595630,28.593607,1.549548,3.711933e+01,1.812476,23.289877,1.513612,20.133837,1.530100,3.956914e+01,1.562351,17.817841,1.303757,2.697609e+01,1.479542,19.543901,1.731311,8.752648,1.490124,11.907196,1.366059,13.101655,1.466991,9.196527,1.701964,11.451623,1.723226,14.925256,1.480636,12.808880,1.547595,8.657048,1.632471,11.605240,1.442562,2.166036e+01,1.300395,13.505555,1.361824,1.758499e+01,1.441426,6.700172,1.770516,14.482513,1.201231,2.303248e+01,1.213803,8.864180,1.38321,19.495246,1.524863,16.430559,1.274409,10.182456,1.795212,16.915841,1.283054,13.189403,1.641368,16.317221,1.710912,11.900601,1.322007,1.713305e+01,1.582910,9.333561,1.423376,20.929370,1.686838,9.933899,1.638675,18.937966,1.405999,12.467400,1.524483,10.690008,1.493357,11.086685,1.282746,11.063649,1.556986,11.965574,1.400936,11.082843,1.413827,18.969167,1.360138,13.388244,1.601238,2.605003e+01,1.319766,15.755147,1.650912,16.320239,1.370736,2.664411e+01,1.322218,19.746608,1.308151,17.801299,1.494488,12.997785,1.387872,24.522245,16.710691,2.033782,11.474681,13.741677,21.438326,16.881584,16.777782,16.143026,16.717333,20.897435,15.961397
std,0.497556,5.436131e+03,0.497211,1186.044698,0.476531,5.922551e+03,0.486076,665.664792,0.499203,510.826242,0.469480,2.625322e+03,0.483607,987.694108,0.494196,5.353557e+03,0.490771,2181.449850,0.497540,4.447033e+03,0.390333,2132.029811,0.499816,629.801897,0.499094,5.281149e+03,0.496098,1921.010776,0.459880,6.286508e+03,0.499583,1530.334325,0.44327

In [7]:
df.sample(10)

,q1,t1,q2,t2,q3,t3,q4,t4,q5,t5,q6,t6,q7,t7,q8,t8,q9,t9,q10,t10,q11,t11,q12,t12,q13,t13,q14,t14,q15,t15,q16,t16,q17,t17,q18,t18,q19,t19,q20,t20,q21,t21,q22,t22,q23,t23,q24,t24,q25,t25,q26,t26,q27,t27,q28,t28,q29,t29,q30,t30,q31,t31,q32,t32,q33,t33,q34,t34,q35,t35,q36,t36,q37,t37,q38,t38,q39,t39,q40,t40,q41,t41,q42,t42,q43,t43,q44,t44,q45,t45,q46,t46,q47,t47,q48,t48,q49,t49,q50,t50,q51,t51,q52,t52,q53,t53,q54,t54,q55,t55,q56,t56,q57,t57,q58,t58,q59,t59,q60,t60,q61,t61,q62,t62,scale_e,scale_i,scale_f,scale_t,scale_s,scale_n,scale_j,scale_p,psychotype
64872,1,6.23,2.0,5.03,1.0,7.51,2.0,6.32,1.0,6.70,2.0,10.52,2.0,5.33,1.0,6.36,2.0,8.96,1.0,6.82,2.0,5.83,2.0,6.81,2.0,9.52,1.0,5.43,1.0,3.60,2.0,5.94,2.0,3.05,2.0,2.28,1.0,2.86,1.0,2.81,1.0,2.90,2.0,3.26,2.0,5.47,1.0,2.78,1.0,2.85,2.0,7.80,2.0,4.18,1.0,2.86,2.0,2.16,2.0,6.34,2.0,3.58,2.0,2.90,2.0,3.39,1.0,3.88,1.0,4.68,1.0,8.54,2.0,2.30,1.0,4.59,1.0,2.41,2.0,2.91,2.0,1.89,2.0,3.43,2.0,3.11,1.0,8.08,2.0,2.93,2.0,6.22,1.0,6.51,2.0,6.29,1.0,7.08,2.0,4.81,2.0,5.40,2.0,4.16,2.0,7.67,2.0,6.68,1.0,8.19,1.0,6.09,2.0,11.38,1.0,10.02,1.0,5.84,1.0,9.72,Не кем,6.11,2.0,5.03,12.0,20.0,8.0,26.0,14.0,19.0,10.0,23.0,INTP
34052,2,5.58,2.0,10.27,1.0,9.72,1.0,14.78,1.0,11.94,2.0,32.30,1.0,6.54,2.0,16.75,2.0,81.11,2.0,47.14,2.0,5.85,1.0,13.34,2.0,24.80,2.0,16.57,2.0,14.39,1.0,12.94,2.0,4.30,1.0,15.35,2.0,5.10,2.0,7.51,1.0,12.72,2.0,4.92,2.0,8.32,1.0,4.85,2.0,122.33,1.0,38.37,1.0,11.33,2.0,7.14,1.0,5.47,2.0,30.59,1.0,7.21,2.0,9.71,1.0,20.29,1.0,25.87,1.0,3.82,2.0,14.97,1.0,10.01,2.0,11.96,1.0,9.03,2.0,16.18,2.0,5.78,1.0,20.82,1.0,4.06,2.0,16.38,2.0,10.07,1.0,5.79,1.0,10.23,1.0,7.02,2.0,27.48,1.0,7.07,2.0,56.73,2.0,20.15,2.0,14.34,2.0,20.02,2.0,9.44,2.0,31.41,1.0,9.30,2.0,16.76,2.0,7.07,1.0,10.21,Дизайнер,18.79,3.0,8.30,12.0,22.0,13.0,16.0,16.0,19.0,25.0,6.0,INTJ
70437,2,6.86,2.0,4.69,1.0,6.64,2.0,16.66,2.0,6.69,1.0,10.30,1.0,13.92,2.0,6.72,2.0,6.30,2.0,42.60,2.0,8.91,1.0,14.20,2.0,10.45,1.0,5.72,1.0,5.93,1.0,5.05,2.0,5.45,1.0,4.60,2.0,4.78,2.0,6.52,2.0,4.58,2.0,5.71,1.0,3.80,2.0,5.72,2.0,3.06,2.0,5.43,1.0,5.86,2.0,4.64,1.0,4.19,1.0,7.48,1.0,6.38,1.0,5.02,1.0,4.90,2.0,3.91,1.0,7.56,2.0,6.55,1.0,4.03,2.0,6.16,2.0,5.55,1.0,5.17,2.0,6.17,2.0,9.98,1.0,4.85,2.0,4.88,1.0,8.31,1.0,5.49,2.0,6.78,1.0,3.73,2.0,6.93,1.0,3.67,1.0,12.16,1.0,5.54,2.0,54.03,1.0,4.97,2.0,6.72,2.0,7.92,1.0,11.97,1.0,8.00,1.0,6.39,2.0,11.31,NaN,3.29,3.0,5.85,5.0,28.0,19.0,19.0,23.0,12.0,32.0,6.0,ISFJ
127145,1,9.83,2.0,16.07,1.0,17.82,1.0,18.77,1.0,31.62,2.0,12.03,1.0,10.27,1.0,9.26,2.0,20.66,1.0,9.48,1.0,26.86,2.0,15.96,2.0,25.32,2.0,11.76,1.0,7.82,2.0,12.05,2.0,11.95,2.0,6.58,1.0,9.22,1.0,13.80,2.0,6.86,2.0,5.80,1.0,15.74,1.0,14.93,1.0,10.47,1.0,11.42,1.0,38.89,1.0,4.04,2.0,2.49,2.0,13.58,1.0,11.99,1.0,7.24,1.0,13.94,1.0,7.38,2.0,11.73,1.0,43.91,1.0,9.42,1.0,8.58,1.0,9.45,1.0,4.92,2.0,17.98,1.0,22.63,1.0,13.26,1.0,7.78,2.0,8.01,1.0,7.58,2.0,87.04,2.0,6.33,1.0,7.52,1.0,7.02,1.0,10.77,1.0,7.10,1.0,13.79,2.0,34.30,2.0,7.30,1.0,11.41,1.0,21.70,1.0,21.27,1.0,17.05,1.0,7.85,никем,7.81,2.0,20.30,25.0,11.0,16.0,26.0,12.0,16.0,16.0,18.0,ENTP
93993,1,6.64,2.0,4.76,2.0,11.43,2.0,5.55,2.0,19.54,2.0,25.75,2.0,9.94,2.0,8.40,1.0,29.16,2.0,66.44,2.0,14.74,1.0,9.05,2.0,7.28,2.0,4.99,1.0,7.49,1.0,7.41,2.0,3.57,1.0,5.53,1.0,8.04,1.0,3.54,2.0,3.60,2.0,3.58,2.0,4.07,2.0,3.90,2.0,2.63,1.0,5.66,1.0,4.01,1.0,2.05,1.0,2.30,2.0,5.99,1.0,20.75,1.0,2.21,2.0,5.43,1.0,4.04,1.0,8.15,2.0,9.44,2.0,3.76,1.0,5.38,2.0,9.37,1.0,3.67,1.0,1.97,2.0,12.49,2.0,6.25,2.0,6.86,2.0,3.06,1.0,5.04,1.0,60.11,1.0,3.88,1.0,4.32,1.0,3.60,1.0,7.28,1.0,16.85,2.0,7.92,1.0,8.87,1.0,7.52,2.0,9.54,2.0,9.93,2.0,6.12,1.0,21.61,1.0,9.39,школьник,13.74,2.0,6.39,18.0,20.0,8.0,21.0,12.0,19.0,16.0,23.0,INTP
5244,2,3.68,2.0,4.41,1.0,4.68,2.0,9.02,1.0,5.25,2.0,16.70,2.0,8.04,2.0,6.58,2.0,7.26,1.0,6.89,2.0,5.30,2.0,6.63,2.0,8.62,2.0,3.46,1.0,3.29,1.0,3.74,2.0,3.68,1.0,3.46,1.0,1.73,1.0,2.21,1.0,2.41,2.0,2.10,2.0,2.41,1.0,3.50,2.0,1.64,1.0,7.24,1.0,4.25,2.0,3.63,1.0,1.57,2.0,4.83,1.0,6.34,1.0,8.60,1.0,5.81,

## Чистка данных

Проверим дубликаты.

In [8]:
df.duplicated().value_counts()

False    194515
True         53
dtype: int64

Сбросим дубликаты.

In [9]:
df.drop_duplicates(inplace=True)

Фактор инфляции дисперсии (VIF) больше 5 говорит о высокой степени коллинеарности. Шкалы `scale_?` получаются путём суммирования результатов ответов на определённые вопросы, поэтому обрадают мультиколлинеарностью и должны быть удалены для обработки линейными моделями. Также удалим психотип, как производное от шкал и ответов.

In [10]:
columns_to_drop = ['scale_e', 'scale_i', 'scale_f', 'scale_t', 'scale_s', 'scale_n', 'scale_j', 'scale_p', 'psychotype']

df.drop(columns=columns_to_drop, axis=1, inplace=True)

Сбросим оставшиеся выявленные пропуски в данных.

In [11]:
df.dropna(inplace=True)

Проверим оставшиеся данные.

In [12]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149700 entries, 93 to 194566
Data columns (total 124 columns):
 #    Column  Non-Null Count   Dtype  
---   ------  --------------   -----  
 0    q1      149700 non-null  int64  
 1    t1      149700 non-null  float64
 2    q2      149700 non-null  float64
 3    t2      149700 non-null  float64
 4    q3      149700 non-null  float64
 5    t3      149700 non-null  float64
 6    q4      149700 non-null  float64
 7    t4      149700 non-null  float64
 8    q5      149700 non-null  float64
 9    t5      149700 non-null  float64
 10   q6      149700 non-null  float64
 11   t6      149700 non-null  float64
 12   q7      149700 non-null  float64
 13   t7      149700 non-null  float64
 14   q8      149700 non-null  float64
 15   t8      149700 non-null  float64
 16   q9      149700 non-null  float64
 17   t9      149700 non-null  float64
 18   q10     149700 non-null  float64
 19   t10     149700 non-null  float64
 20   q11     149700 non-null

Избавимся от строк, где ответ на вопрос об удовлетворённости неопределённый.

In [13]:
df.drop(df[df['q62'] == 3].index, inplace=True)

Создадим для дальнейшей работы список колонок со временем.

In [14]:
time_columns = []

for i in range(1, 63):
    time_columns.append('t'+str(i))

Сбросим строки, в которых время оказалось отрицательным.

In [15]:
for time_column in time_columns:
    df.drop(df[df[time_column] <= 0].index, inplace=True)

Проверим результат.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94137 entries, 94 to 194566
Columns: 124 entries, q1 to t62
dtypes: float64(122), int64(1), object(1)
memory usage: 89.8+ MB


Избавимся от экстремальных значений, отбросив 0.1 и 99.9 персентили.

In [17]:
indexes_to_drop = set()

for time_column in time_columns:
    q_min = df[time_column].quantile(.001)
    q_max = df[time_column].quantile(.999)
    indexes_to_drop.update(df[df[time_column] <= q_min].index)
    indexes_to_drop.update(df[df[time_column] >= q_max].index)
    
df.drop(list(indexes_to_drop), inplace=True)
df.reset_index(drop=True, inplace=True)
print(q_min)
print(q_max)

0.9613599999999999
166.0024000000056


Проверим общее оставшееся количество строк.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88744 entries, 0 to 88743
Columns: 124 entries, q1 to t62
dtypes: float64(122), int64(1), object(1)
memory usage: 84.0+ MB


Поменяем типы колонок.

In [19]:
# uint8 колонки
uint8_columns = set()
for i in range(1, 61):
    uint8_columns.add('q'+str(i))
uint8_columns.add('q62')

# изменяем dtypes колонок
df = df.astype({
    **{_:'uint8' for _ in list(uint8_columns)}
})

Проверим результат.

In [20]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88744 entries, 0 to 88743
Data columns (total 124 columns):
 #    Column  Non-Null Count  Dtype  
---   ------  --------------  -----  
 0    q1      88744 non-null  uint8  
 1    t1      88744 non-null  float64
 2    q2      88744 non-null  uint8  
 3    t2      88744 non-null  float64
 4    q3      88744 non-null  uint8  
 5    t3      88744 non-null  float64
 6    q4      88744 non-null  uint8  
 7    t4      88744 non-null  float64
 8    q5      88744 non-null  uint8  
 9    t5      88744 non-null  float64
 10   q6      88744 non-null  uint8  
 11   t6      88744 non-null  float64
 12   q7      88744 non-null  uint8  
 13   t7      88744 non-null  float64
 14   q8      88744 non-null  uint8  
 15   t8      88744 non-null  float64
 16   q9      88744 non-null  uint8  
 17   t9      88744 non-null  float64
 18   q10     88744 non-null  uint8  
 19   t10     88744 non-null  float64
 20   q11     88744 non-null  uint8  
 21   t11     88

А также посмотрим статистическую информацию по полученному датасету.

In [21]:
df.describe()

,q1,t1,q2,t2,q3,t3,q4,t4,q5,t5,q6,t6,q7,t7,q8,t8,q9,t9,q10,t10,q11,t11,q12,t12,q13,t13,q14,t14,q15,t15,q16,t16,q17,t17,q18,t18,q19,t19,q20,t20,q21,t21,q22,t22,q23,t23,q24,t24,q25,t25,q26,t26,q27,t27,q28,t28,q29,t29,q30,t30,q31,t31,q32,t32,q33,t33,q34,t34,q35,t35,q36,t36,q37,t37,q38,t38,q39,t39,q40,t40,q41,t41,q42,t42,q43,t43,q44,t44,q45,t45,q46,t46,q47,t47,q48,t48,q49,t49,q50,t50,q51,t51,q52,t52,q53,t53,q54,t54,q55,t55,q56,t56,q57,t57,q58,t58,q59,t59,q60,t60,t61,q62,t62
count,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000,88744.000000
mean,1.522255,11.670955,1.559553,10.591019,1.347066,10.634785,1.368926,9.198865,1.446070,8.917394,1.667369,13.818304,1.381479,10.847737,1.577447,12.172707,1.545772,11.613510,1.552409,12.052298,1.801057,10.605497,1.550527,12.412982,1.535247,13.765837,1.563700,8.697879,1.302477,7.781034,1.487717,8.509629,1.747566,6.897066,1.506626,6.518209,1.362402,6.526738,1.491177,6.225146,1.709332,6.550841,1.715586,6.286744,1.480348,6.610523,1.532735,6.346055,1.635435,6.536768,1.402450,8.006971,1.280301,7.376361,1.364554,6.715795,1.466465,5.697635,1.770091,8.738388,1.196994,7.753855,1.192486,7.209273,1.366211,9.535758,1.514559,7.149922,1.254834,7.072110,1.785552,9.550308,1.277698,6.852061,1.638409,9.920997,1.694931,7.816363,1.327628,7.274940,1.538087,7.004153,1.396579,12.125042,1.717029,7.909178,1.655650,10.518058,1.399193,7.715451,1.493183,7.636718,1.488844,7.443634,1.269596,8.020684,1.570957,7.401604,1.399565,7.873256,1.396838,11.543212,1.367304,7.167911,1.620876,11.800815,1.327909,9.718428,1.639773,10.874782,1.331470,10.044081,1.318838,12.072309,1.316866,11.558583,1.481024,9.294475,1.406247,11.484264,12.427217,1.391102,7.254294
std,0.499507,37.498066,0.496444,15.881627,0.476040,13.825612,0.482517,10.223205,0.497086,12.418026,0.471158,22.765280,0.485753,15.020169,0.493968,20.962527,0.497903,18.015887,0.497248,16.881615,0.399207,16.114768,0.497443,18.219828,0.498759,20.702596,0.495928,10.172231,0.459333,9.340046,0.499852,10.770531,0.434411,7.490578,0.499959,7.312440,0.480697,6.407131,0.499925,7.078157,0.454073,7.512421,0.451137,6.768454,0.499616,7.913435,0.498930,7.866204,0.481311,7.437074,0.490394,7.815392,0.449149,9.657449,0.481308,9.015758,0.498877,6.094055,0.420776,9.594719,0.397730,9.938074,0.394255,7.941591,0.481771,10.698081,0.499791,9.394272,0.435770,7.591577,0.410441,11.278331,0.447866,7.415714,0.480464,12.440270,0.460439,9.026456,0.469351,8.891307,0.498550,7.497039,0.489190,17.134165,0.450445

## Обработка столбца с профессиями

Осмотрим текущие данные.

In [22]:
with np.printoptions(threshold=np.inf):
    print(df['q61'].unique())

['Данный момент не работаю ' 'Учитель' 'я школьник' 'рекламист'
 'Руководитель участка' 'Повар' 'Фитнес '
 'Менеджер регионального отдела продаж' 'инженер' 'преподаватель'
 'акушерка' 'Учителем ' 'Ведущий менеджер ' 'Свое дело.' 'Курьер'
 'Студент ' 'предприниматель ' 'Бизнесмен' 'Пожарный' 'Разнорабочий'
 'учусь ' 'дизайнер' 'Педагогом организатором' 'Врач' 'Бухгалтер '
 'Бухгалтер' 'Инженер по охране труда' 'никем' 'Домохозяйка' 'Военный'
 'Продавец' 'Авиадиспетчер' 'танцор' 'Оптик' 'Педагог' 'Не работаю'
 'Кадровик' 'Технолог' 'менеджер' 'не кем' 'некем ' 'Кладовщик '
 'Архивариус ' 'мастер ногтевого сервиса' 'Кладовщик' 'Юрист' 'учусь'
 'Лоо' 'бухгалтером' 'Студент' 'Регистратоп' 'Ведущий специалист '
 'студент' 'Программист' 'Дизайнер' 'Администратор ' 'Никем'
 'Рабочая производства' 'На складе' 'архитектор' 'Оператор харвестера'
 'УЧУСЬ' 'Политиком' 'Начальник' 'Медсестра' 'Секретарь корректор'
 'дизайнер мебели' 'Психологом' 'официант' 'Начальник отдела гсм' 'Тренер'
 'Врачом' '

Загрузим словарь библиотеки SymSpell.

In [23]:
sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)

dictionary_path = pkg_resources.resource_filename('symspellpy', 'ru-100k.txt')

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1, separator=' ', encoding='UTF-8')

True

Создадим функцию для фильтрации и исправления опечаток в графе `профессия`.

In [24]:
morph = pymorphy2.MorphAnalyzer()
stops = set(stopwords.words("russian"))
lemmas_list = []

def review_to_wordlist(to_review: str) -> list:
    '''
    Функция принимает строку с текстом. Удаляет всё кроме букв и пробелов. Переводит в нижний регистр.
    Разделяет на отдельные слова. Отсеивает слова входящие в список стоп-слов. Выделяет из полученных слов леммы
    и возвращает в виде списка слов.
    '''
    # убираем все символы кроме букв и пробелов
    review_text = re.sub('[^а-яА-Яa-zA-Z]', ' ', to_review)
    # переводим в нижний регистр и разделяем на отдельные слова
    words = review_text.lower().split()
    # отсеиваем стоп-слова
    words = [w for w in words if not w in stops]
    # выделяем леммы
    words = [morph.parse(w)[0].normal_form for w in words]
    # проверяем каждое слово на ошибки модулем SymSpellPy
    result = []
    for word in words:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
        try:
            result.append(suggestions[0].term)
            lemmas_list.append(suggestions[0].term)
        except:
            None
    
    return(result)

Применим полученную функцию для очистки данных.

In [25]:
df['reviewed'] = df['q61'].transform(review_to_wordlist)

Проверим результат.

In [26]:
df.sample(10)

,q1,t1,q2,t2,q3,t3,q4,t4,q5,t5,q6,t6,q7,t7,q8,t8,q9,t9,q10,t10,q11,t11,q12,t12,q13,t13,q14,t14,q15,t15,q16,t16,q17,t17,q18,t18,q19,t19,q20,t20,q21,t21,q22,t22,q23,t23,q24,t24,q25,t25,q26,t26,q27,t27,q28,t28,q29,t29,q30,t30,q31,t31,q32,t32,q33,t33,q34,t34,q35,t35,q36,t36,q37,t37,q38,t38,q39,t39,q40,t40,q41,t41,q42,t42,q43,t43,q44,t44,q45,t45,q46,t46,q47,t47,q48,t48,q49,t49,q50,t50,q51,t51,q52,t52,q53,t53,q54,t54,q55,t55,q56,t56,q57,t57,q58,t58,q59,t59,q60,t60,q61,t61,q62,t62,reviewed
66377,1,9.8400,2,17.4900,2,13.6900,2,9.1100,2,14.0200,2,18.5000,2,16.4100,2,14.2000,2,11.7400,2,15.7600,2,10.9400,2,14.2100,1,15.4700,2,9.8500,2,7.1200,2,6.9800,1,4.1900,1,3.9800,1,8.0400,1,3.6700,2,5.0900,1,6.9000,2,4.3500,2,9.0400,2,6.7500,2,11.6600,2,6.1200,1,3.0800,2,4.8600,2,11.5500,1,7.1900,1,3.7100,1,20.0300,1,4.3700,1,6.4600,2,19.0000,1,4.9300,1,9.1900,1,8.5300,1,3.7500,2,4.3800,1,4.7900,2,4.3000,2,9.3500,1,6.8000,1,5.5000,1,12.9900,2,7.4000,1,5.4200,1,6.1700,1,3.3800,1,4.0700,2,19.0900,2,5.6900,1,18.1200,1,15.9200,1,17.0400,1,11.1700,1,13.1800,1,8.5200,Бомжом,9.0000,1,9.0000,[том]
6151,1,3.5200,1,2.4800,1,4.2300,1,5.0800,2,1.9100,2,5.0200,1,4.9200,1,3.0800,1,3.9100,1,5.7200,2,5.1100,2,7.9900,1,6.6600,2,1.9000,2,1.8500,2,1.9400,1,1.6900,1,2.2500,1,2.6900,1,1.3400,1,1.8900,2,1.5800,2,1.5300,2,3.1400,1,2.4900,1,2.4700,2,1.9500,1,2.6700,1,1.5200,1,2.1800,2,3.0700,1,2.3300,2,4.1300,1,3.2000,1,1.5500,2,3.5600,1,2.3700,2,4.6300,2,2.8700,2,3.2800,1,3.6000,1,4.4400,1,3.2900,2,5.3200,2,3.2100,1,3.0500,2,2.8800,1,5.2100,2,2.5800,2,2.0600,2,9.4000,1,3.3800,2,10.4500,1,5.3600,1,2.2700,1,4.5500,1,6.2900,1,3.0700,1,3.8700,1,6.9500,менеджер продаж,28.8800,2,3.6200,"[менеджер, продажа]"
7043,2,24.1500,2,27.1600,1,21.0500,1,28.1800,1,32.3600,2,25.5600,1,9.7900,2,17.4400,2,29.3800,1,9.3000,2,65.1500,2,14.1600,2,20.0400,2,15.8900,1,9.8100,1,12.3600,2,9.0800,2,15.4600,1,10.7800,2,44.1300,2,8.8500,2,7.1700,2,9.2600,2,7.3700,2,25.5300,1,25.2300,1,11.2300,1,8.6600,2,5.7600,2,15.5200,2,15.5500,1,17.9300,1,33.5600,2,16.7400,1,16.6700,2,16.0500,2,24.8400,2,22.4000,2,14.3100,1,20.9100,1,29.2000,1,14.4700,2,13.1700,1,19.8100,1,38.0000,1,11.5100,2,15.9400,1,39.9400,2,33.3300,1,25.7100,2,10.4400,2,8.9900,1,42.2300,2,25.7200,1,17.5200,2,80.3300,1,49.8700,2,28.3400,1,47.1600,2,74.4800,Продажник,17.3300,1,12.7300,[продажи]
49547,1,4.4100,1,9.1700,1,8.4300,1,6.8000,2,5.3300,2,10.9000,1,4.5200,1,7.8500,1,7.7700,1,8.5200,2,5.6300,2,8.3100,2,8.3300,2,4.9200,1,6.0700,2,5.4000,1,5.0300,2,3.2000,1,3.9000,1,4.2500,1,6.9500,2,2.3800,1,5.5100,1,5.1100,1,2.6600,2,8.1400,2,3.9300,1,4.6300,1,2.7500,2,6.5900,1,4.9600,1,4.6000,1,7.4800,2,5.7400,1,3.5500,1,5.6600,1,2.6800,1,4.4300,2,9.4600,2,4.9000,1,5.0400,2,6.7600,2,6.9900,2,5.2600,1,8.1100,2,5.0700,1,4.3000,1,4.8600,2,5.2700,2,12.2200,1,13.0200,2,6.6600,2,9.4700,1,11.2100,1,6.9100,2,8.2400,1,5.2300,2,14.6500,1,6.1800,2,6.4900,Тетапрактик,8.1800,1,3.4700,[тетапрактик]
87359,1,8.4375,1,8.4375,2,8.4375,2,8.4375,2,8.4375,2,8.4375,1,8.4375,2,8.4375,1,8.4375,1,8.4375,2,8.4375,2,8.4375,2,8.4375,2,8.4375,1,8.4375,2,8.4375,2,8.4375,2,8.4375,1,8.4375,1,8.4375,2,8.4375,2,8.4375,2,8.4375,1,8.4375,1,8.4375,2,8.4375,2,8.4375,1,8.4375,2,8.4375,2,8.4375,1,8.4375,1,8.4375,1,8.4375,1,8.4375,1,8.4375,2,8.4375,1,8.4375,1,8.4375,1,8.4375,1,8.4375,1,8.4375,1,8.4375,2,8.4375,2,8.4375,2,8.4375,1,8.4375,2,8.4375,2,8.4375,1,8.4375,2,8.4375,2,8.4375,1,8.4375,1,8.4375,2,8.4375,2,8.4375,1,8.4375,1,8.4375,2,8.4375,1,8.4375,1,8.4375,"Коммуникация, продажи | Менеджер по продажам",8.4375,2,8.4375,"[коммуникация, продажа, менеджер, продажа]"
52230,1,2.4500,1,1.4500,1,2.0700,2,2.4000,1,1.7000,2,8.9800,2,3.8200,1,2.4500,1,3.8200,2,3.2800,2,1.9500,2,2.3500,1,5.4100,1,1.2600,1,1.3200,2,1.5600,2,1.3900,2,2.0200,1,2.1000,1,2.0900,1,1.4400,2,1.4800,2,2.1200,1,1.2600,1,1.6400,1,2.5100,1,2.2300,1,2.2700,2,3.3500,2,5.5600,1,2.3000,2,4.9200,1,5.7500,2,3.0600,1,3.0800,1,8.7600,2,3.6000,2,10.7000,1,3.7800,2,2.7800,1,3.2100,1,4.8300,2,9.8500,1,11.2400,1,4.0000,1,4.5500,2,6.1000,1,2.

In [27]:
with np.printoptions(threshold=np.inf):
    print(Counter(lemmas_list).most_common())

[('менеджер', 6991), ('руководитель', 6426), ('учиться', 6055), ('никто', 5425), ('студент', 5084), ('продажа', 3704), ('школьник', 3635), ('специалист', 3232), ('ученик', 3061), ('организация', 2387), ('коммуникация', 2210), ('персонал', 2138), ('бухгалтер', 2026), ('отдел', 1988), ('продавец', 1945), ('оператор', 1784), ('инженер', 1606), ('дизайн', 1602), ('директор', 1542), ('школа', 1506), ('учитель', 1480), ('юрист', 1408), ('торговля', 1300), ('посредничество', 1292), ('администратор', 1259), ('финансы', 1250), ('реклама', 1217), ('обучение', 1185), ('начальник', 1129), ('программист', 1095), ('художник', 1088), ('и', 1073), ('работа', 1055), ('маркетинг', 1022), ('кто', 1019), ('врач', 956), ('звено', 955), ('среднее', 954), ('it', 953), ('работать', 945), ('экономист', 921), ('психолог', 830), ('клиент', 817), ('консультант', 815), ('повар', 660), ('преподаватель', 656), ('мастер', 614), ('предприниматель', 593), ('администрирования', 583), ('домохозяйка', 567), ('старший', 56

Попробуем отсечь те леммы что встречаются реже 4 раз.

In [28]:
with np.printoptions(threshold=np.inf):
    print(Counter(lemmas_list).most_common()[:-2680:-1])

[('ними', 1), ('автостраховщик', 1), ('vfhrtnjkju', 1), ('никому', 1), ('северный', 1), ('авторитет', 1), ('криминальной', 1), ('мер', 1), ('борец', 1), ('музейщик', 1), ('джигит', 1), ('отдавать', 1), ('heart', 1), ('startup', 1), ('фабрикантов', 1), ('гостей', 1), ('agile', 1), ('фотосессия', 1), ('инж', 1), ('фельдъегерь', 1), ('достойный', 1), ('камин', 1), ('каменный', 1), ('фитосанитарный', 1), ('шов', 1), ('пппаап', 1), ('price', 1), ('конфликт', 1), ('урегулирование', 1), ('теория', 1), ('реселлера', 1), ('революционера', 1), ('визитом', 1), ('руководительница', 1), ('экструдорщик', 1), ('бригады', 1), ('шиномонтажный', 1), ('интеллект', 1), ('ростовщик', 1), ('груди', 1), ('путешественника', 1), ('арендодатель', 1), ('кардиология', 1), ('фасовщица', 1), ('копья', 1), ('nikem', 1), ('они', 1), ('безроба', 1), ('арбитражный', 1), ('дендролог', 1), ('киберкотлета', 1), ('ламимейкер', 1), ('инвестиций', 1), ('kadrov', 1), ('otdel', 1), ('шкодьникнуть', 1), ('дьявола', 1), ('вестни

In [29]:
print('6557')

6557


In [30]:
pd.Series(lemmas_list).nunique()

4655

Составим список лемм, строки с которыми необходимо сбросить.

In [31]:
lemmas_to_drop = {'учиться', 'никто', 'студент', 'школьник', 'ученик', 'школа', 'безработный',
                 'студентка', 'некого', 'и', 'кто', 'пока', 'ах', 'хотеть', 'тат'}
for lemma, freq in Counter(lemmas_list).most_common()[:-2680:-1]:
    lemmas_to_drop.add(lemma)

Сбросим строки со строками содержащими леммы для сброса.

In [36]:
indexes_to_drop = set()

def drop_lemmas(row: pd.Series, lemmas: list) -> None:
        '''
        Функция проверяет содержится ли в столбце 'reviwed' леммы из списка.
        Если верно - добавляет индекс в лист для сброса.
        Также добавит в индекс пустые списки, те в которых были только стоп слова.
        '''
        for word in row['reviewed']:
            if word in lemmas:
                indexes_to_drop.add(row.name)
        if len(row['reviewed']) == 0:
            indexes_to_drop.add(row.name)
        return None

In [37]:
df.apply(drop_lemmas, lemmas=lemmas_to_drop, axis=1)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
31       None
32       None
33       None
34       None
35       None
36       None
37       None
38       None
39       None
40       None
41       None
42       None
43       None
44       None
45       None
46       None
47       None
48       None
49       None
50       None
51       None
52       None
53       None
54       None
55       None
56       None
57       None
58       None
59       None
60       None
61       None
62       None
63       None
64       None
65       None
66       None
67       None
68       None
69       None
70       None
71    

In [38]:
df.drop(list(indexes_to_drop), inplace=True)
df.reset_index(drop=True, inplace=True)

Вновь проверим леммы что вошли в усечённый набор.

In [39]:
lemmas_list = []

def count_lemmas(row: pd.Series) -> None:
    '''
    Функция для проверки текущего суммарного состава лемм в столбце.
    '''
    for word in row['reviewed']:
        lemmas_list.append(word)
    return None

df.apply(count_lemmas, axis=1)
    
with np.printoptions(threshold=np.inf):
    print(Counter(lemmas_list).most_common())

print(pd.Series(lemmas_list).nunique())

[('менеджер', 6499), ('руководитель', 6336), ('продажа', 3692), ('специалист', 3111), ('организация', 2384), ('коммуникация', 2210), ('бухгалтер', 2024), ('продавец', 1921), ('отдел', 1901), ('персонал', 1874), ('оператор', 1732), ('инженер', 1587), ('дизайн', 1555), ('директор', 1514), ('учитель', 1461), ('юрист', 1393), ('торговля', 1299), ('посредничество', 1292), ('финансы', 1249), ('администратор', 1241), ('реклама', 1120), ('программист', 1076), ('художник', 1058), ('начальник', 1031), ('работа', 1004), ('it', 943), ('врач', 932), ('маркетинг', 928), ('обучение', 925), ('экономист', 919), ('среднее', 898), ('звено', 898), ('работать', 814), ('клиент', 814), ('консультант', 803), ('психолог', 802), ('повар', 650), ('преподаватель', 643), ('предприниматель', 589), ('домохозяйка', 565), ('администрирования', 560), ('хозяйство', 551), ('коммерческий', 550), ('сельский', 545), ('старший', 544), ('бизнес', 538), ('грузчик', 535), ('официант', 529), ('школьника', 516), ('мастер', 506), 